<a href="https://colab.research.google.com/github/LucaPamboerSamhoud/RankGPT/blob/main/RankGPTRoleExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch
!pip install beir
!git clone https://github.com/LucaPamboerSamhoud/RankGPT.git
!pip install -r RankGPT/requirements.txt
!pip install faiss-cpu
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=e4feba6afa49e3dec9dbd40ee88c5264454d89508ef78255a2468726022604d5
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for p

In [ ]:
!pip install litellm==1.53.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 27.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:

import pandas as pd
import numpy as np
corpus = pd.read_csv('./RankGPT/NovelEval/corpus.tsv', sep='\t', header=None)
queries = pd.read_csv('./RankGPT/NovelEval/queries.tsv', sep='\t', header=None)
qrles = pd.read_csv('./RankGPT/NovelEval/qrels.txt', sep='\s', header=None)

<ipython-input-3-bb135fba60ae>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  qrles = pd.read_csv('./RankGPT/NovelEval/qrels.txt', sep='\s', header=None)


In [ ]:
corpus.head()


,0,1
0,0-0,Spider-Man: Across the Spider-Verse is a 2023 ...
1,0-1,Sony began developing a sequel to Into the Spi...
2,0-2,Spider-Man: Across the Spider-Verse premiered ...
3,0-3,Spider-Man: Into the Spider-Verse arrived in t...
4,0-4,"""The exact number? Oh boy, we kept adding, lik..."


In [ ]:
queries.head()

,0,1
0,0,How many different Spider-Men are there in Acr...
1,1,What is the screen resolution of vision pro?
2,2,Which film was the 2023 Palme d'Or winner?
3,3,Who will be the CEO of Twitter after Elon Musk...
4,4,How many goals did Haaland scored in the 2023 ...


In [ ]:
corpus[['query', 'index']] = corpus[0].str.split('-', expand=True)

In [ ]:
# Initialize the list to store results
rank_results = []

# Iterate through each query
for _, query_row in queries.iterrows():
    query_id = query_row[0]
    query_text = query_row[1]
    # Create the item dictionary for the current query
    item = {
        'query': query_text,
        'hits': []
    }

    # Find all related contents based on query index
    for _, content_row in corpus.iterrows():
        if str(content_row['query']) == str(query_id):
            hit = {
                'content': content_row[1]
            }
            item['hits'].append(hit)

    # Append the result for the current query
    rank_results.append(item)

In [ ]:
import sys
sys.path.append("/content/RankGPT")

In [ ]:
from rank_gpt import permutation_pipeline, sliding_windows

api_key=""

## Analysis

### Ranking with old role

#### Reranking

In [ ]:
# Run sliding window permutation generation
new_results_baseline = []
for item in rank_results:
    role = "an intelligent assistant that can rank passages based on their relevancy to the query"
    new_item = sliding_windows(item, rank_start=0, rank_end=20, window_size=10, step=5, model_name='huggingface/meta-llama/Llama-3.2-3B-Instruct', api_key=api_key, role=role)
    new_results_baseline.append(new_item)

### Ranking with one size fits all role

In [ ]:
# Run sliding window permutation generation
new_results_one_role = []
for item in rank_results:
    query = item['query']
    role = "a general expert reviewer - knowledgeable on all kinds of topics- that can rank passages based on their relevancy to the query"
    new_item = sliding_windows(item, rank_start=0, rank_end=20, window_size=10, step=5, model_name='huggingface/meta-llama/Llama-3.2-3B-Instruct', api_key=api_key, role=role)
    new_results_one_role.append(new_item)

### Ranking with role per query

In [ ]:
query_roles_dict = {
    "How many different Spider-Men are there in Across the Spider-Verse?": "a film scholar specializing in superhero and animated movies",
    "What is the screen resolution of vision pro?": "a technology expert specializing in cutting-edge consumer electronics",
    "Which film was the 2023 Palme d'Or winner?": "a film awards analyst with expertise in global cinema accolades",
    "Who will be the CEO of Twitter after Elon Musk is no longer the CEO?": "a corporate strategist specializing in leadership trends and executive transitions",
    "How many goals did Haaland scored in the 2023 Champions League Final": "a sports analyst specializing in football performance statistics",
    "Where did Benzema go after leaving Real Madrid?": "a football transfer expert specializing in player moves and contracts",
    "Where was the 2023 Premier League FA Cup Final held?": "a football journalist specializing in event coverage and match locations",
    "What is the name of the combined Deepmind and Google Brain?": "an AI research analyst specializing in AI organizations and collaborations",
    "Where will Blackpink's 2023 world tour concert in France be held?": "a music industry analyst with expertise in global concert tours",
    "Where did the G7 Summit 2023 take place?": "an event location expert specializing in international political and diplomatic gatherings",
    "What are the best papers of CVPR 2023?": "a computer vision researcher specializing in top-tier academic conferences",
    "What is the release date of song Middle Ground?": "a music trends analyst focusing on new releases and chart performances",
    "Who wins NBA Finals 2023?": "a basketball analyst with expertise in league championships and player statistics",
    "Who does Momoa play in Fast X?": "a movie analyst specializing in blockbuster franchises and character arcs",
    "What is Messi's annual income after transferring to Miami?": "a sports business and finance expert specializing in player contracts, sponsorships, and earnings",
    "Who sang the theme song of Transformers Rise of the Beasts?": "a film music analyst specializing in soundtracks for major franchises",
    "How much video memory does the DGX GH200 have?": "a computer hardware expert specializing in high-performance GPUs",
    "What are the new features of PyTorch 2?": "a deep learning framework expert specializing in PyTorch advancements",
    "Who is the villain in The Flash?": "a film analyst specializing in superhero movies and plot analysis",
    "Who win 2023 Laureus World Sportsman Of The Year Award?": "a sports journalist specializing in global sports awards",
    "The Little Mermaid first week box office?": "an entertainment industry analyst specializing in box office trends and film performances"
}

# Run sliding window permutation generation
new_results_role_specific = []
for item in rank_results:
    query = item['query']
    role = query_roles_dict.get(query, "an intelligent assistant")
    role = role+" that can rank passages based on their relevancy to the query"
    new_item = sliding_windows(item, rank_start=0, rank_end=20, window_size=10, step=5, model_name='huggingface/meta-llama/Llama-3.2-3B-Instruct', api_key=api_key, role=role)
    new_results_role_specific.append(new_item)

## Analysis on the results

In [ ]:
corpus_raw = pd.read_csv('./RankGPT/NovelEval/corpus.tsv', sep='\t', header=None)
qrels = pd.read_csv('./RankGPT/NovelEval/qrels.txt', sep='\s', header=None)


<ipython-input-26-001d5d25de14>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  qrels = pd.read_csv('./RankGPT/NovelEval/qrels.txt', sep='\s', header=None)


In [ ]:
import pandas as pd
import numpy as np

# Function to preprocess the dataframes and merge them
def preprocess_data(new_results, corpus, qrels):
    reranked_hits = pd.DataFrame(new_results, columns=['query', 'hits'])
    df_exploded = reranked_hits.explode('hits', ignore_index=True)
    df_exploded['hits'] = df_exploded['hits'].apply(lambda x: x.get('content') if isinstance(x, dict) else None)

    # Load and preprocess the corpus
    corpus['rel_score'] = qrels[3].values
    df_exploded = df_exploded.rename(columns={'hits': 'content'})

    # Merge the DataFrame
    df_merged = df_exploded.merge(corpus, left_on='content', right_on=1, how='left')
    df_merged = df_merged.drop(columns=['content'])  # Optional: Drop redundant column

    return df_merged

# Function to calculate DCG
def calculate_DCG(relevance_scores):
    dcg = 0
    for i, rel in enumerate(relevance_scores, start=1):
        dcg += rel / np.log2(i + 1)
    return dcg

# Function to calculate nDCG
def calculate_nDCG(relevance_scores):
    dcg_score = calculate_DCG(relevance_scores)
    ideal_relevance_scores = sorted(relevance_scores, reverse=True)
    idcg_score = calculate_DCG(ideal_relevance_scores)

    return 0 if idcg_score == 0 else dcg_score / idcg_score

# Main function to compute average nDCG
def compute_average_nDCG(df_merged, top_k=10):
    ndcg_scores = []

    # Iterate over each unique query
    for query in df_merged['query'].unique():
        query_data = df_merged[df_merged['query'] == query]

        # Extract the top K relevance scores
        top_k_relevance = query_data['rel_score'].head(top_k)
        ndcg = calculate_nDCG(top_k_relevance)
        ndcg_scores.append(ndcg)

    # Calculate the average nDCG across all queries
    return np.mean(ndcg_scores)

# Function to compute nDCG scores per query
def compute_ndcg_per_query(df_merged, top_k):
    query_ndcg_scores = []
    for query in df_merged['query'].unique():
        query_data = df_merged[df_merged['query'] == query]
        top_k_relevance = query_data['rel_score'].head(top_k)
        ndcg = calculate_nDCG(top_k_relevance)
        query_ndcg_scores.append({'query': query, f'nDCG@{top_k}': ndcg})
    return pd.DataFrame(query_ndcg_scores)


In [ ]:
# Define the conditions and results in a list of dictionaries
results = [
    {
        'Condition': 'Baseline',
        'nDCG@10': compute_average_nDCG(preprocess_data(new_results_baseline, corpus_raw, qrels), top_k=10),
        'nDCG@5': compute_average_nDCG(preprocess_data(new_results_baseline, corpus_raw, qrels), top_k=5),
        'nDCG@1': compute_average_nDCG(preprocess_data(new_results_baseline, corpus_raw, qrels), top_k=1)
    },
    {
        'Condition': 'One Role',
        'nDCG@10': compute_average_nDCG(preprocess_data(new_results_one_role, corpus_raw, qrels), top_k=10),
        'nDCG@5': compute_average_nDCG(preprocess_data(new_results_one_role, corpus_raw, qrels), top_k=5),
        'nDCG@1': compute_average_nDCG(preprocess_data(new_results_one_role, corpus_raw, qrels), top_k=1)
    },
    {
        'Condition': 'Specific Roles',
        'nDCG@10': compute_average_nDCG(preprocess_data(new_results_role_specific, corpus_raw, qrels), top_k=10),
        'nDCG@5': compute_average_nDCG(preprocess_data(new_results_role_specific, corpus_raw, qrels), top_k=5),
        'nDCG@1': compute_average_nDCG(preprocess_data(new_results_role_specific, corpus_raw, qrels), top_k=1)
    }
]

# Convert the results into a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)

        Condition   nDCG@10    nDCG@5    nDCG@1
0        Baseline  0.861694  0.907710  0.857143
1        One Role  0.858983  0.885609  0.904762
2  Specific Roles  0.875699  0.916186  0.857143


In [ ]:
# Function to compute nDCG scores per query
def compute_ndcg_per_query(df_merged, top_k):
    query_ndcg_scores = []
    for query in df_merged['query'].unique():
        query_data = df_merged[df_merged['query'] == query]
        top_k_relevance = query_data['rel_score'].head(top_k)
        ndcg = calculate_nDCG(top_k_relevance)
        query_ndcg_scores.append({'query': query, f'nDCG@{top_k}': ndcg})
    return pd.DataFrame(query_ndcg_scores)

# Preprocess data for each condition
df_merged_baseline = preprocess_data(new_results_baseline, corpus_raw, qrels)
df_merged_one_role = preprocess_data(new_results_one_role, corpus_raw, qrels)
df_merged_role_specific = preprocess_data(new_results_role_specific, corpus_raw, qrels)

# Compute nDCG scores for each top_k and align them by merging
def create_ndcg_dataframe(df_merged):
    ndcg1_df = compute_ndcg_per_query(df_merged, top_k=1)
    ndcg5_df = compute_ndcg_per_query(df_merged, top_k=5)
    ndcg10_df = compute_ndcg_per_query(df_merged, top_k=10)

    # Merge dataframes to align by query
    ndcg_df = ndcg1_df.merge(ndcg5_df, on='query', how='outer')
    ndcg_df = ndcg_df.merge(ndcg10_df, on='query', how='outer')
    return ndcg_df

# Create separate DataFrames for each condition
baseline_ndcg_df = create_ndcg_dataframe(df_merged_baseline)
one_role_ndcg_df = create_ndcg_dataframe(df_merged_one_role)
specific_roles_ndcg_df = create_ndcg_dataframe(df_merged_role_specific)


In [ ]:
# Function to compute all nDCG scores for a condition and add roles
def compute_all_ndcg_with_roles(df_merged, condition_name, query_roles_dict):
    query_ndcg_scores = []
    for query in df_merged['query'].unique():
        query_data = df_merged[df_merged['query'] == query]
        relevance_scores = query_data['rel_score']

        # Retrieve the role for the query (default to None if not found)
        role = query_roles_dict.get(query, None)

        # Compute nDCG scores
        scores = {
            'query': query,
            'role': role,
            f'{condition_name}_nDCG@1': calculate_nDCG(relevance_scores.head(1)),
            f'{condition_name}_nDCG@5': calculate_nDCG(relevance_scores.head(5)),
            f'{condition_name}_nDCG@10': calculate_nDCG(relevance_scores.head(10)),
        }
        query_ndcg_scores.append(scores)
    return pd.DataFrame(query_ndcg_scores)

# Preprocess data for each condition
df_merged_baseline = preprocess_data(new_results_baseline, corpus_raw, qrels)
df_merged_one_role = preprocess_data(new_results_one_role, corpus_raw, qrels)
df_merged_role_specific = preprocess_data(new_results_role_specific, corpus_raw, qrels)

# Compute results for each condition
baseline_ndcg_df = compute_all_ndcg_with_roles(df_merged_baseline, 'Baseline', query_roles_dict)
one_role_ndcg_df = compute_all_ndcg_with_roles(df_merged_one_role, 'OneRole', query_roles_dict)
specific_roles_ndcg_df = compute_all_ndcg_with_roles(df_merged_role_specific, 'SpecificRoles', query_roles_dict)

# Merge results into a single DataFrame
combined_df = baseline_ndcg_df.merge(one_role_ndcg_df, on=['query', 'role'], how='outer')
combined_df = combined_df.merge(specific_roles_ndcg_df, on=['query', 'role'], how='outer')

# Display the final combined DataFrame
combined_df

,query,role,Baseline_nDCG@1,Baseline_nDCG@5,Baseline_nDCG@10,OneRole_nDCG@1,OneRole_nDCG@5,OneRole_nDCG@10,SpecificRoles_nDCG@1,SpecificRoles_nDCG@5,SpecificRoles_nDCG@10
0,How many different Spider-Men are there in Acr...,a film scholar specializing in superhero and a...,1.0,1.000000,0.792865,1.0,0.906025,0.906025,0.0,0.732829,0.732829
1,How many goals did Haaland scored in the 2023 ...,a sports analyst specializing in football perf...,1.0,0.967468,0.917655,1.0,0.946902,0.900547,1.0,0.981572,0.910098
2,How much video memory does the DGX GH200 have?,a computer hardware expert specializing in hig...,1.0,0.981572,0.922887,1.0,0.981572,0.922887,1.0,0.981572,0.922887
3,The Little Mermaid first week box office?,an entertainment industry analyst specializing...,1.0,1.000000,0.976782,1.0,1.000000,0.982238,1.0,1.000000,0.976782
4,What are the best papers of CVPR 2023?,a computer vision researcher specializing in t...,1.0,0.950234,0.950234,1.0,0.950234,0.853943,1.0,0.950234,0.853943
5,What are the new features of PyTorch 2?,a deep learning framework expert specializing ...,1.0,0.881199,0.873351,1.0,0.881199,0.876193,1.0,0.951776,0.941394
6,What is Messi's annual income after transferri...,a sports business and finance expert specializ...,1.0,0.877215,0.812653,1.0,0.850345,0.792087,1.0,0.946902,0.946902
7,What is the name of the combined Deepmind and ...,an AI research analyst specializing in AI orga...,1.0,1.000000,0.944051,1.0,0.965195,0.897220,1.0,1.000000,0.967338
8,What is the release date of song Middle Ground?,a music trends analyst focusing on new release...,1.0,0.987696,0.876183,1.0,0.987696,0.876183,1.0,0.987696,0.902043
9,What is the screen resolution of vision pro?,a technology expert specializing in cutting-ed...,1.0,0.967468,0.940276,1.0,0.955830,0.943355,1.0,0.967468,0.940276


## Combining three runs together for significance tests

In [ ]:
combined_df.to_csv('results iteration 3.csv')

In [ ]:
df1 = pd.read_csv('results iteration 1.csv').drop(columns=['Difference'])
df2 = pd.read_csv('results iteration 2.csv')
df3 = pd.read_csv('results iteration 3.csv')

# Add the iteration column to each dataframe
df1['iteration'] = 1
df2['iteration'] = 2
df3['iteration'] = 3

# Merge all dataframes
merged_df = pd.concat([df1, df2, df3], ignore_index=True).drop(columns= ['Unnamed: 0'])

# Display the merged dataframe
merged_df

,query,role,Baseline_nDCG@1,Baseline_nDCG@5,Baseline_nDCG@10,OneRole_nDCG@1,OneRole_nDCG@5,OneRole_nDCG@10,SpecificRoles_nDCG@1,SpecificRoles_nDCG@5,SpecificRoles_nDCG@10,iteration
0,How many different Spider-Men are there in Acr...,film analyst - a professional who specializes ...,1.0,0.919721,0.851959,1.0,1.000000,1.000000,0.0,0.732829,0.732829,1
1,How many goals did Haaland scored in the 2023 ...,football analyst – a professional who speciali...,1.0,0.967468,0.917655,0.0,0.655591,0.648769,0.0,0.693426,0.687148,1
2,How much video memory does the DGX GH200 have?,hardware engineer - a professional with expert...,1.0,0.981572,0.922887,1.0,0.981572,0.922887,0.0,0.748010,0.747950,1
3,The Little Mermaid first week box office?,entertainment industry journalist – a journali...,1.0,1.000000,0.982238,1.0,1.000000,1.000000,1.0,0.919721,0.877500,1
4,What are the best papers of CVPR 2023?,computer vision researcher – an expert in the ...,1.0,0.950234,0.950234,1.0,0.950234,0.950234,1.0,0.900313,0.900313,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Who is the villain in The Flash?,a film analyst specializing in superhero movie...,1.0,0.830450,0.820699,1.0,0.881199,0.857214,1.0,0.893535,0.846766,3
59,Who sang the theme song of Transformers Rise o...,a film music analyst specializing in soundtrac...,0.0,0.430677,0.448644,0.0,0.000000,0.388958,0.0,0.624051,0.624051,3
60,Who will be the CEO of Twitter after Elon Musk...,a corporate strategist specializing in leaders...,1.0,1.000000,0.966223,1.0,0.965195,0.934994,1.0,1.000000,0.966223,3
61,Who win 2023 Laureus World Sportsman Of The Ye...,a sports journalist specializing in global spo...,1.0,0.977724,0.938685,1.0,1.000000,0.952730,1.0,0.977724,0.938685,3


In [ ]:
merged_df.to_csv('results.csv')

## Compute score per query

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon

# Load the final DataFrame
df = merged_df

# Step 1: Reshape the DataFrame for easier analysis
# Create a pivot table for nDCG@10
ndcg_10 = df.pivot_table(
    index=["query", "iteration"],
    values=["Baseline_nDCG@5", "OneRole_nDCG@5", "SpecificRoles_nDCG@5"]
)

# Drop rows with missing values
ndcg_10 = ndcg_10.dropna()

# Rename columns for simplicity
ndcg_10.columns = ['baseline', 'one_role', 'specific_roles']

# Step 2: Perform Wilcoxon tests and calculate differences and p-values
results = []

# Group by query to perform pairwise comparisons at the condition level
for query, group in ndcg_10.groupby(level=0):  # Group by query
    baseline = group['baseline']
    one_role = group['one_role']
    specific_roles = group['specific_roles']

    # Initialize result dictionary for each query
    query_results = {"query": query}

    # Baseline vs One Role
    abs_diff_br = baseline.mean() - one_role.mean()
    if not (baseline == one_role).all():
        stat_br, p_val_br = wilcoxon(baseline, one_role)
        query_results["baseline_vs_one_role_diff"] = abs_diff_br
        query_results["baseline_vs_one_role_p_value"] = p_val_br
        query_results["baseline_vs_one_role_p<0.05"] = p_val_br < 0.05
    else:
        query_results["baseline_vs_one_role_diff"] = 0
        query_results["baseline_vs_one_role_p_value"] = "No differences"
        query_results["baseline_vs_one_role_p<0.05"] = "No differences"

    # One Role vs Specific Roles
    abs_diff_or = one_role.mean() - specific_roles.mean()
    if not (one_role == specific_roles).all():
        stat_or, p_val_or = wilcoxon(one_role, specific_roles)
        query_results["one_role_vs_specific_roles_diff"] = abs_diff_or
        query_results["one_role_vs_specific_roles_p_value"] = p_val_or
        query_results["one_role_vs_specific_roles_p<0.05"] = p_val_or < 0.05
    else:
        query_results["one_role_vs_specific_roles_diff"] = 0
        query_results["one_role_vs_specific_roles_p_value"] = "No differences"
        query_results["one_role_vs_specific_roles_p<0.05"] = "No differences"

    # Baseline vs Specific Roles
    abs_diff_bs = baseline.mean() - specific_roles.mean()
    if not (baseline == specific_roles).all():
        stat_bs, p_val_bs = wilcoxon(baseline, specific_roles)
        query_results["baseline_vs_specific_roles_diff"] = abs_diff_bs
        query_results["baseline_vs_specific_roles_p_value"] = p_val_bs
        query_results["baseline_vs_specific_roles_p<0.05"] = p_val_bs < 0.05
    else:
        query_results["baseline_vs_specific_roles_diff"] = 0
        query_results["baseline_vs_specific_roles_p_value"] = "No differences"
        query_results["baseline_vs_specific_roles_p<0.05"] = "No differences"

    # Append results for this query
    results.append(query_results)

# Convert results to a DataFrame for easy viewing
results_df = pd.DataFrame(results)

# Display the summarized table
pd.set_option('display.max_columns', None)  # Ensure all columns are visible
print(results_df)


                                                query  \
0   How many different Spider-Men are there in Acr...   
1   How many goals did Haaland scored in the 2023 ...   
2      How much video memory does the DGX GH200 have?   
3           The Little Mermaid first week box office?   
4              What are the best papers of CVPR 2023?   
5             What are the new features of PyTorch 2?   
6   What is Messi's annual income after transferri...   
7   What is the name of the combined Deepmind and ...   
8     What is the release date of song Middle Ground?   
9        What is the screen resolution of vision pro?   
10    Where did Benzema go after leaving Real Madrid?   
11           Where did the G7 Summit 2023 take place?   
12  Where was the 2023 Premier League FA Cup Final...   
13  Where will Blackpink's 2023 world tour concert...   
14         Which film was the 2023 Palme d'Or winner?   
15                     Who does Momoa play in Fast X?   
16                   Who is the

/usr/local/lib/python3.10/dist-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


In [ ]:
results_df.to_csv('results_wilcoxon_per_query.csv')

## Compute score per condition

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon

# Load the final DataFrame
df = merged_df

# Step 1: Ensure the relevant columns are numeric
columns_to_convert = ["Baseline_nDCG@10", "OneRole_nDCG@10", "SpecificRoles_nDCG@10"]
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Step 2: Compute the average scores across all queries for each condition and iteration
avg_scores = df.groupby("iteration")[columns_to_convert].mean()

# Extract averaged scores for each condition
baseline = avg_scores["Baseline_nDCG@10"]
one_role = avg_scores["OneRole_nDCG@10"]
specific_roles = avg_scores["SpecificRoles_nDCG@10"]

# Step 3: Perform Wilcoxon tests and calculate differences and p-values
results = []

# Baseline vs One Role
abs_diff_br = abs(baseline.mean() - one_role.mean())
stat_br, p_val_br = wilcoxon(baseline, one_role)
results.append({
    "comparison": "Baseline vs One Role",
    "abs_diff": abs_diff_br,
    "p_value": p_val_br,
    "p<0.05": p_val_br < 0.05
})

# One Role vs Specific Roles
abs_diff_or = abs(one_role.mean() - specific_roles.mean())
stat_or, p_val_or = wilcoxon(one_role, specific_roles)
results.append({
    "comparison": "One Role vs Specific Roles",
    "abs_diff": abs_diff_or,
    "p_value": p_val_or,
    "p<0.05": p_val_or < 0.05
})

# Baseline vs Specific Roles
abs_diff_bs = abs(baseline.mean() - specific_roles.mean())
stat_bs, p_val_bs = wilcoxon(baseline, specific_roles)
results.append({
    "comparison": "Baseline vs Specific Roles",
    "abs_diff": abs_diff_bs,
    "p_value": p_val_bs,
    "p<0.05": p_val_bs < 0.05
})

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Display the table
pd.set_option('display.max_columns', None)  # Ensure all columns are visible
print(results_df)


                   comparison  abs_diff  p_value  p<0.05
0        Baseline vs One Role  0.002872     0.25   False
1  One Role vs Specific Roles  0.008505     0.50   False
2  Baseline vs Specific Roles  0.005633     0.50   False


In [ ]:
results_df.to_csv('results_wilcoxon_average.csv')